In [5]:
import boto3
import pandas as pd
s3 = boto3.client('s3')

with open('balanced_reviews.csv', 'wb') as data:
    s3.download_fileobj('dlfinalproject', 'balanced_reviews.csv', data)
df = pd.read_csv('balanced_reviews.csv')

In [6]:
s3 = boto3.resource('s3')
s3.meta.client.upload_file('my project_LSTMs_CNN.ipynb', 'dlfinalproject', 'my project.ipynb')

In [7]:
df.head()

,Unnamed: 0,business_id,cool,date,funny,review_id,stars,text,useful,user_id,fun_bin
0,17,cHdJXLlKNWixBXpDwEGb_A,1,2015-04-01 16:30:00,7,6BnQwlxRn7ZuWdzninM9sQ,3,I love chinese food and I love mexican food. W...,1,JSrP-dUmLlwZiI7Dp3PQ2A,1
1,21,Mem13A3C202RzT53npn4NA,9,2017-05-13 10:41:43,6,IPw8yWiyqnfBzzWmypUHgg,5,If you are looking for the best pierogies in P...,9,5JVY32_bmTBfIGpCCsnAfw,1
2,62,SU56w479vUfFHsvmvQIf7A,6,2016-07-25 03:55:20,5,E4LqIZ7DJd_R4ZHSNKx4RQ,4,So good! They didn't make it to 5 stars due to...,7,DoRCeCcJbrsM2BiAKj3trA,1
3,126,tjAeaGdxf7I4xN9M7wGJNQ,4,2014-07-13 14:32:56,5,TaoaX7MqCujFRNaJBns2Sw,5,While the prices are a bit high for a make-you...,8,x37OyP--VEFE5p-xreplYA,1
4,246,FhIeCF6QrsLaRvAeu0oEPQ,4,2013-06-24 06:42:29,5,3Qc49B7dA0ONmCxrn5iwCQ,2,OVERALL: The food isn't good (I explain below)...,13,2k8OVAPxlXHsA5X6EIoQpQ,1


In [8]:
import os
import re
import string

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

'''from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
'''
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
df_reviews = df[['funny','text', 'fun_bin']]

In [11]:
pd.set_option('display.max_colwidth', -1)
df_reviews.head()

,funny,text,fun_bin
0,7,"I love chinese food and I love mexican food. What can go wrong? A couple of things. First things first, this place is more of a ""rice bowl"" kind of place. I thought it was going to be more diverse as far as the menu goes, but its mainly rice bowls you get with different kinds of meats. The ordering was a little confusing at first, but one of the employees helped us out and I got the 2-item bowl and got the jade chicken and hengrenade chicken with all rice(jerk). I also ordered a jade chicken quesadilla on the side.\n\nI'm gonna admit, this place looks kinda dirty. I don't think Arizona uses those health department letter grade system like California does, but if I were to just judge by how it looked inside, i'd give it a ""C"" grade lol. We waited for about 15 minutes or so and finally got our food. We took it to go and ate at our hotel room. \n\nMmmm... the food was just alright. The jade chicken was nothing special. It tasted like any generic chinese fast food orange chicken/sesame chicken variant. The hengrenade chicken, although was the less spicier version of the jerk chicken, was still pretty spicy for me. Just be warned the jerk chicken is super spicy. If you aren't sure, ask for a sample at the restaurant before ordering, but it was way too spicy for me. \n\nThe jade chicken quesadilla was decent, but nothing special. Just imagine orange chicken in between a tortilla and cheese. A friend of mine ordered a jade chicken burrito and we were confused when we pulled it out of the bag because it was literally the size of Mcdonald's apple pie. If you order the burrito, be warned that it's a burrito for gnomes and smurfs, but he said it was tasty. \n\nThey provide a snicker doodle sugar cookie for each meal and it was decent, again nothing special. \n\nNot gonna lie, the next day my stomach felt like a little mexican dude and chinese dude were wrestling and throwing molotov cocktails inside. I used the bathroom like 5 times. I don't recommend eating this place if you have a lot to do the next day.",1
1,6,"If you are looking for the best pierogies in Pittsburgh, this is your place. There are a few small tables outside but most of the business is carry out. Pierogies Plus wins Best Pierogies every year. Why? Because the owner is from Poland and she is making the real deal pierogies. The best part is that they are hand pinched by a group of older Polish and Hungarian women. \nThe biggest seller is potato and cheese but they sell many flavors. They are like plump pillows of softness. You can buy them buy the dozen. You can get them cold to take home and freeze or warm and ready to eat. The warm ones are served with butter and onions. It's definitely a comfort food. The best part is that they ship internationally. Yes, they are that good.",1
2,5,"So good! They didn't make it to 5 stars due to the prices are a bit high for the amount of food and the location is a bit unsavory. \nThe decor and atmosphere was surprisingly nice, from the outside I expected to be more run down inside. The staff was very nice. We were surprised how empty the dining room was for a Friday evening.\nWe got Vegetable Samosas to start then ordered Chicken Tikka Masala, Lamb Rogan Josh, rice and plain Naan. Our only complaint was the lamb could've been more tender but everything was flavorful and delicious. \nI would definitely go again if given the chance.",1
3,5,"While the prices are a bit high for a make-your-own pizza, the taste makes up for it. I love going to Seventh Street market, sitting Not Just Coffee and having a drink while waiting for delicious fresh made pizza from Pure. I've taken this to go as well as eaten inside the market, and I can say that the pizza doesn't do well reheated. So try to eat it fresh while there if possible.\n\nIf one of their specialty pizzas sounds good to you, go for it, as those are definitely a better deal for the amount of toppings you get for the money. I wanted what I wanted, though, so I en

In [8]:
import pandas as pd
import numpy as np
#import nltk
#from nltk.corpus import stopwords
#from nltk.stem import SnowballStemmer

from string import punctuation


import numpy as np 
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
#from spacymoji import Emoji

In [9]:
#Run two commands below and then restart the kernel
#!pip install --upgrade pip

    100% |████████████████████████████████| 1.4MB 6.1MB/s ta 0:00:01
fastai 1.0.52 requires nvidia-ml-py3, which is not installed.
thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.0 which is incompatible.
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1


In [10]:
#!pip install -U spacy[cuda92]

     |████████████████████████████████| 29.8MB 59.7MB/s eta 0:00:01
     |████████████████████████████████| 3.2MB 44.5MB/s eta 0:00:01
     |████████████████████████████████| 184kB 52.8MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 45.2MB/s eta 0:00:01
     |████████████████████████████████| 491kB 48.2MB/s eta 0:00:01
     |████████████████████████████████| 320.7MB 153kB/s  eta 0:00:01     |████████████████████████████▎   | 283.9MB 1.7MB/s eta 0:00:22     |█████████████████████████████▍  | 294.6MB 23.0MB/s eta 0:00:02
  Stored in directory: /home/ec2-user/.cache/pip/wheels/eb/ba/a3/9af9f326ed0d75a4540378af64a05a0e42be39d9b8513f3aea
Successfully built thinc-gpu-ops
ERROR: fastai 1.0.52 requires nvidia-ml-py3, which is not installed.
  Found existing installation: thinc 6.12.1
    Uninstalling thinc-6.12.1:
      Successfully uninstalled thinc-6.12.1
  Found existing installation: spacy 2.0.18
    Uninstalling spacy-2.0.18:
      Successfully uninstalled spacy-2.0.18


In [12]:
!python3 -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [13]:
import spacy

spacy.prefer_gpu()
spacy.load("en_core_web_sm")

In [14]:
import pandas as pd
import numpy as np
#import nltk
#from nltk.corpus import stopwords
#from nltk.stem import SnowballStemmer

from string import punctuation


import numpy as np 
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 

In [15]:

re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
def sub_br(x): return re_br.sub("\n", x)

#nlp = spacy.load("en")
#spacy.load('en')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS



def clean_text(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
   # text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=\(\)]", " ", text) # keep punctuatuin, numnbers and letters
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " . ", text) #Add space to the dot
    text = re.sub(r"!", " ! ", text) #Add space to the exclamation sign
    text = re.sub(r":", " :", text) #Add space before : sign
    text = re.sub(r";", " ;", text) #Add space before ; sign
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    #text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    #text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    # find emojis
    emoji_list = []
    '''
    for word in text.split():
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)
    emoji_list'''
    #text = text.split()

    return text

my_tok = spacy.load('en_core_web_sm')
#emoji = Emoji(my_tok)
#my_tok.add_pipe(emoji, first=True)
def spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(clean_text(x))]

def remove_stop_words(tokens): return [tok for tok in tokens if tok not in spacy_stopwords]


### Building a vocabulary

In [16]:
from collections import Counter
counts = Counter()
for sent in df_reviews['text']:
    try:
        counts.update(remove_stop_words(spacy_tok(sent)))
    except:
        pass

In [17]:
counts

Counter({'love': 32008,
         'chinese': 7274,
         'food': 133459,
         'mexican': 6873,
         '.': 1974798,
         'wrong': 8025,
         '?': 76651,
         'couple': 10174,
         'things': 14292,
         'place': 106803,
         '"': 129623,
         'rice': 20425,
         'bowl': 9057,
         'kind': 12303,
         'thought': 13658,
         'going': 23998,
         'diverse': 493,
         'far': 9426,
         'menu': 44809,
         'goes': 3716,
         'mainly': 795,
         'bowls': 1969,
         'different': 14945,
         'kinds': 1378,
         'meats': 3860,
         'ordering': 7141,
         'little': 36265,
         'confusing': 710,
         'employees': 3579,
         'helped': 1852,
         'got': 45468,
         '2': 31328,
         '-': 245468,
         'item': 3670,
         'jade': 172,
         'chicken': 47015,
         'hengrenade': 4,
         'rice(jerk': 1,
         ')': 123431,
         'ordered': 46914,
         'quesadil

### Vocabulary

In [18]:
# Vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [19]:
# WHat is the 99% quantile of  length of the sentence?

df_reviews['len_text'] = df_reviews['text'].apply(lambda x: len(x.split()))


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [20]:
df_reviews['len_text'].quantile(0.95)

496.0

In [21]:
# note that spacy_tok takes a while run it just once
def encode_sentence(sent, vocab2index, N=500, padding_start=True):
    "Encoding a sentence adding padding"
    x = remove_stop_words(spacy_tok(sent))
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in x])
    l = min(N, len(enc1))
    if padding_start:
        enc[:l] = enc1[:l]
    else:
        enc[N-l:] = enc1[:l]
    return enc, l

### Splitting into train and validation sets

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df_reviews['text'], df_reviews['fun_bin'], test_size=0.2, random_state=42)
    

In [23]:
X_train.reset_index(inplace=True, drop=True)
X_valid.reset_index(inplace=True, drop=True)

In [24]:
y_train.reset_index(inplace=True, drop=True)
y_valid.reset_index(inplace=True, drop=True)

In [25]:
y_train.shape

(120401,)

### Writing a dataset

In [26]:
class YelpDataset(Dataset):
    def __init__(self, df, y, N=400, padding_start=True):
        self.df = df
        self.X = [encode_sentence(sent, vocab2index, N, padding_start) for sent in self.df]
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x, s = self.X[idx]
        return x, s, self.y[idx]


In [27]:
train_ds =  YelpDataset(X_train, y_train, padding_start=False)
valid_ds =  YelpDataset(X_valid, y_valid, padding_start=False)


In [28]:
neg=[]
i=0
for x,s,y in train_ds:
    if s <=0:
        neg.append(i)
    i+=1

neg

[118582]

In [29]:
X_train.loc[118582,]

'Neither'

In [30]:
X_train.drop(index = 118582, inplace=True)
y_train.drop(index = 118582, inplace=True)
X_train.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)

In [31]:
batch_size = 1000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [39]:
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))


### LSTM V0

In [61]:
class LSTMV0Model(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(LSTMV0Model,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        out_pack, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [62]:
def train_epocs_v0(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            # s is not used in this model
            x = x.long().cuda()
            y = y.float().cuda()
            y_pred = model(x)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics_v0(model, valid_dl)
        if i % 5 == 1:
            print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [63]:
def val_metrics_v0(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in valid_dl:
        # s is not used here
        x = x.long().cuda()
        y = y.float().cuda().unsqueeze(1)
        y_hat = model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [68]:
vocab_size = len(words)
print(vocab_size)
model_v0 = LSTMV0Model(vocab_size, 100, 100).cuda()

133541


In [69]:
train_epocs_v0(model_v0, epochs=20, lr=0.01)

train loss 0.480 val loss 0.531 and val accuracy 0.727
train loss 0.426 val loss 0.573 and val accuracy 0.738
train loss 0.467 val loss 0.551 and val accuracy 0.748
train loss 0.417 val loss 0.570 and val accuracy 0.750


In [70]:
save_model(model_v0,"lstmv0_lr0.01_20")

In [71]:
model_v0 = LSTMV0Model(vocab_size, 100, 100).cuda()
train_epocs_v0(model_v0, epochs=10, lr=0.005)

train loss 0.495 val loss 0.511 and val accuracy 0.752
train loss 0.413 val loss 0.509 and val accuracy 0.772


In [73]:
save_model(model_v0,"lstmv0_lr0.005_10")

In [74]:
model_v0 = LSTMV0Model(vocab_size, 100, 100).cuda()
train_epocs_v0(model_v0, epochs=20, lr=0.001)

train loss 0.537 val loss 0.974 and val accuracy 0.558
train loss 0.465 val loss 0.718 and val accuracy 0.634
train loss 0.431 val loss 0.911 and val accuracy 0.601
train loss 0.407 val loss 0.968 and val accuracy 0.600


In [75]:
save_model(model_v0,"lstmv0_lr0.001_20")

### LSTM v1: Model with variable length

In [76]:
# dataset with padding at the end
train_ds_2 =  YelpDataset(X_train, y_train, padding_start=True)
valid_ds_2 =  YelpDataset(X_valid, y_valid, padding_start=True)


In [77]:
class LSTMModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(LSTMModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(0.5)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        
    def forward(self, x, s):
        # sorting
        s, sort_index = torch.sort(s.float(), 0,descending=True) # s is the length of the sentence. Sort these lengths
        s = s.numpy().tolist() # 
        x = x[sort_index]
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True) # We want LSTM to forget the padding, but in order to apply 
        #ordering mini batches withtin the model
        out_pack, (ht, ct) = self.lstm(x_pack) 
        out = self.linear(ht[-1]) # Problem here is that output is not sorted! 
        return torch.zeros_like(out).scatter_(0, sort_index.unsqueeze(1).cuda(), out) # scatter_ is undoing the sorting with the given sorting index
        # kind of sorting back with the original indexing
    


In [78]:
def train_epocs(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.long().cuda()
            y = y.float().cuda()
            y_pred = model(x, s)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [79]:
def val_metrics(model, val_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in val_dl:
        x = x.long().cuda()
        y = y.float().cuda().unsqueeze(1)
        y_hat = model(x, s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [80]:
batch_size = 5000
train_dl = DataLoader(train_ds_2, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds_2, batch_size=batch_size)

In [81]:
vocab_size = len(words)
print(vocab_size)
modelv1 = LSTMModel(vocab_size, 100, 100).cuda()

133541


In [82]:
train_epocs(modelv1, epochs=20, lr=0.01)

train loss 0.603 val loss 0.591 and val accuracy 0.726
train loss 0.530 val loss 0.520 and val accuracy 0.758
train loss 0.434 val loss 0.524 and val accuracy 0.741
train loss 0.393 val loss 0.494 and val accuracy 0.769


In [83]:
save_model(modelv1,"lstmv1_lr0.01_20")

In [84]:

train_epocs(modelv1, epochs=20, lr=0.005)

train loss 0.355 val loss 0.494 and val accuracy 0.777
train loss 0.333 val loss 0.515 and val accuracy 0.771
train loss 0.310 val loss 0.525 and val accuracy 0.771
train loss 0.287 val loss 0.556 and val accuracy 0.772


In [85]:
save_model(modelv1, 'lstmv1_lr0.005_20')

In [86]:
train_epocs(modelv1, epochs=20, lr=0.001)

train loss 0.267 val loss 0.577 and val accuracy 0.766
train loss 0.263 val loss 0.583 and val accuracy 0.766
train loss 0.257 val loss 0.588 and val accuracy 0.766
train loss 0.253 val loss 0.603 and val accuracy 0.766


In [87]:
save_model(modelv1, 'lstmv1_lr0.001_20')

### CNN

In [32]:

class CNNModel(nn.Module):
    
    def __init__(self, vocab_size, hidden_dim):
        super(CNNModel, self).__init__()

        self.embedding = nn.Embedding(vocab_size, hidden_dim, padding_idx=0)
    
        self.conv_3 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=3)
        self.conv_4 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=4)
        self.conv_5 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=5)
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1,2)
        x3 = F.relu(self.conv_3(x))
        x4 = F.relu(self.conv_4(x))
        x5 = F.relu(self.conv_5(x))
        x3 = nn.MaxPool1d(kernel_size = 398)(x3)
        x4 = nn.MaxPool1d(kernel_size = 397)(x4)
        x5 = nn.MaxPool1d(kernel_size = 396)(x5)
        out = torch.cat([x3, x4, x5], 2)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        return self.fc(out)

In [30]:
vocab_size = len(words)
print(vocab_size)
model = CNNModel(vocab_size, 300).cuda()

133541


In [ ]:
batch_size = 1000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [32]:
# testing the model
for x,s,y in train_dl:
    print(x.shape)
    x = x.long().cuda()

torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000, 400])
torch.Size([1000

In [33]:
y_hat = model(x)
y_hat.size()

torch.Size([400, 1])

In [44]:
def val_metrics(m, val_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in val_dl:
        x = x.long().cuda()
        y = y.float().cuda().unsqueeze(1)
        y_hat = model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [45]:
def train_epocs(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.long().cuda()
            y = y.float().cuda()
            y_pred = model(x)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, val_dl)
        #if i % 1 == 1:
        print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [35]:
batch_size = 500
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [37]:
vocab_size = len(words)
print(vocab_size)

133541


In [38]:
model = CNNModel(vocab_size, 300).cuda()
train_epocs(model, epochs=20, lr=0.005 )

train loss 0.471 val loss 0.475 and val accuracy 0.778
train loss 0.171 val loss 1.093 and val accuracy 0.745
train loss 0.083 val loss 2.173 and val accuracy 0.732
train loss 0.073 val loss 3.152 and val accuracy 0.729


In [40]:
save_model(model, 'cnn_lr0.005_20.pth')

In [46]:
model = CNNModel(vocab_size, 300).cuda()
train_epocs(model, epochs=5, lr=0.005 )

train loss 0.057 val loss 5.499 and val accuracy 0.721
train loss 0.063 val loss 6.818 and val accuracy 0.723
train loss 0.057 val loss 6.528 and val accuracy 0.724
train loss 0.052 val loss 6.233 and val accuracy 0.721
train loss 0.059 val loss 6.430 and val accuracy 0.726


In [48]:
save_model(model, 'cnn_lr0.005_5.pth')

In [51]:
model = CNNModel(vocab_size, 300).cuda()
train_epocs(model, epochs=5, lr=0.001)

train loss 0.559 val loss 0.489 and val accuracy 0.768
train loss 0.493 val loss 0.472 and val accuracy 0.779
train loss 0.459 val loss 0.464 and val accuracy 0.781
train loss 0.429 val loss 0.465 and val accuracy 0.782
train loss 0.398 val loss 0.470 and val accuracy 0.782


In [52]:
save_model(model, 'cnn_lr0.001_5.pth')

In [53]:
class CNNModel2(nn.Module):
    
    def __init__(self, vocab_size, hidden_dim):
        super(CNNModel2, self).__init__()

        self.embedding = nn.Embedding(vocab_size, hidden_dim, padding_idx=0)
    
        self.conv_3 = nn.Conv1d(in_channels=hidden_dim, out_channels=150, kernel_size=30)
        self.conv_4 = nn.Conv1d(in_channels=hidden_dim, out_channels=150, kernel_size=40)
        #self.conv_5 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=5)
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1,2)
        x3 = F.relu(self.conv_3(x))
        x4 = F.relu(self.conv_4(x))
       # x5 = F.relu(self.conv_5(x))
        x3 = nn.MaxPool1d(kernel_size = 371)(x3)
        x4 = nn.MaxPool1d(kernel_size = 361)(x4)
       # x5 = nn.MaxPool1d(kernel_size = 396)(x5)
        out = torch.cat([x3, x4], 2)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        return self.fc(out)

In [54]:
batch_size = 500
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [55]:
vocab_size = len(words)
print(vocab_size)
model2 = CNNModel2(vocab_size, 300).cuda()

133541


In [56]:
train_epocs(model2, epochs=10, lr=0.01)

train loss 0.727 val loss 0.470 and val accuracy 0.782
train loss 0.458 val loss 0.470 and val accuracy 0.782
train loss 0.384 val loss 0.470 and val accuracy 0.782
train loss 0.304 val loss 0.470 and val accuracy 0.782
train loss 0.242 val loss 0.470 and val accuracy 0.782
train loss 0.200 val loss 0.470 and val accuracy 0.782
train loss 0.187 val loss 0.470 and val accuracy 0.782
train loss 0.168 val loss 0.470 and val accuracy 0.782
train loss 0.172 val loss 0.470 and val accuracy 0.782
train loss 0.165 val loss 0.470 and val accuracy 0.782


In [59]:
save_model(model2, 'cnn2_lr0.01_10.pth')